*** Download a FASTA file containing the protein gene models for the spotted gar
genome, transfer to HPC ***

*** Construct a BLAST database for each of the protein sets using makeblastdb ***

*** Execute BLASTP using gar proteins as the query and puffer as the subject
database. Do it for all pairs. ***